In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

In [ ]:
categories = ["COVID-19", "NEUMONIA", "NORMAL"]

data = []
label = []

for category in categories:
    for file in os.listdir(category):
        dir = category + "/" + file
        img = imread(dir, as_gray=True)
        img = resize(img, (256, 256))
        data.append(img.flatten())
        label.append(categories.index(category))

In [ ]:
data = np.asarray(data)
label = np.asarray(label)

x_train, x_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, shuffle = True, stratify = label, random_state = 42)

In [ ]:
# Definimos un diccionario que especifica las combinaciones de hiperparámetros a explorar
param_grid = {
    'C': [0.1, 1, 10, 15],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 1, 10]
}

# Creamos un modelo SVC
grid_svc = SVC()

# Creamos un objeto GridSearch para la busqueda de parametros y ajustamos a los datos de entrenamiento
grid_search = GridSearchCV(grid_svc, param_grid, cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Obtenemos los mejores hiperparametros encontrados
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Imprimimos el mejor modelo junto a su puntuacion
print("Model:", best_params)
print("Best Score:", best_score)